# Selection des gares à étudier

__Auteur :__ 

Steve Caron

__Présentation :__ 

Ce script consulte toutes les gares disponibles sur l'API SNCF et en récupère des informations.

Il stocke les résultats dans un fichier : data/liste_gares.csv

__Inputs :__ 

Un fichier .env contenant la clé API stocké sous le nom de variable API_KEY

Un fichier data/liste_gares.csv contenant les informations sur les gares (Créé à partir de gares.ipynb)

__Params :__ 

LISTE_RESEAUX_ETUDIES : Liste des id_reseau que je souhaite inclure dans l'étude

In [1]:
import requests
import os
import pandas as pd
import json
from time import sleep
from dotenv import load_dotenv

Liste des réseaux que je veux étudier

In [14]:
LISTE_RESEAUX_ETUDIES =["network:SNCF:DBS",
                        "network:SNCF:EUROSTAR",
                        "network:SNCF:IC",
                        "network:SNCF:ICN",
                        "network:SNCF:FR:Branding::a80c322c-12b1-4273-9413-e2df5dd6dce8:",
                        "network:SNCF:TGVOUIGO",
                        "network:SNCF:OUIGO_TC",
                        "network:SNCF:SNF",
                        "network:SNCF:TER",
                        "network:SNCF:FR:Branding::6dfb1ec4-50b5-42a8-a531-a792b1ea6f2e:",
                        "network:SNCF:FR:Branding::5c5f14b9-bff2-4f94-8853-0f621bf1a1cf:",
                        "network:SNCF:OUI",
                        "network:SNCF:LYR",
                        "network:SNCF:TN",
                        ]

In [4]:
def requete_reseau(code,api_key):
    '''Requête API permettant de récupérer la liste des réseaux exploités sur une gare'''
    reponse = requests.get(f"https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:{code}/networks", auth=(api_key,""))
    sleep(1)
    return reponse.json()

In [2]:
def requete_frequentation_gare(page):
    ''' Requête API permettant de récupérer les chiffres de fréquentation de toutes les gares'''
    url = f"https://ressources.data.sncf.com/api/explore/v2.1/catalog/datasets/frequentation-gares/records?limit=100&offset={page*100}"
    reponse = requests.get(url).json()
    return reponse

In [6]:
def enregistrer_json(data,nom_fichier):
    '''Cette fonction enregistre un fichier json'''
    with open(nom_fichier,"w") as fc:
        json.dump(data,fc)

In [7]:
def calcul_nombre_page():
    '''Cette fonction calcule le nombre de pages nécessaires pour obtenir toutes les informations sur la fréquentation des gares '''
        
    premiere_requete = requete_frequentation_gare(0)
    nb_gare = premiere_requete["total_count"]
    nb_page = nb_gare // 100
    if nb_gare % 100 != 0:
        nb_page += 1

    return nb_page

In [8]:
def recuperer_toutes_pages(nb_pages):
    '''Cette fonction fait une requête api pour chaque page et stocke les résultats dans une liste'''
    #Initie liste gare
    liste_info_gare = []
    # Requete api pour chaque page
    for page in range(nb_pages):
        #recupère les données de la clé results
        data_page = requete_frequentation_gare(page)["results"]
        #ajoute les données à la liste, element par element
        for item in data_page:
            liste_info_gare.append(item)
    return liste_info_gare  

In [3]:
def extraction_network(data):
    '''Cette fonction extrait les valeurs stockées sous la clé "networks" d'un dictionnaire. Si il n'y a pas de clé, la fonction retourne None'''
    try:
        # Extraction de tous les reseau d'une gare
        tous_reseaux = data["networks"]
        return tous_reseaux

    except KeyError:
        return None


In [10]:
def ajout_reseau(data,cle_api):
    '''Creer une liste de gares en ajoutant un dictionnaire réseau à chaque gare'''

    compteur = 0
    nb_gare = len(data)
    liste_gares_reseaux = []

    for item in data:
        # Extraction du code uic
        code_gare = item["code_uic"]
        # Requete pour récupérer les infos réseau
        resultat = requete_reseau(code_gare,cle_api)
        # Extraction des réseaux
        liste_reseaux = extraction_network(resultat)
        # Ajout d'un ensemble clé valeur avec la liste des réseaux
        item["networks"]=liste_reseaux
        # Ecriture de toutes les informations de la gare dans une liste
        liste_gares_reseaux.append(item)
        #Incrémentation du compteur
        compteur += 1
        # Si le compteur est multiple de 10 affiche un message
        if compteur % 10 == 0:
                print(f"gare n°{compteur} sur {nb_gare}")

    
    return liste_gares_reseaux


In [11]:
def trie_gare(data,LISTE_RESEAUX_ETUDIES):
    '''Cette fonction trie une liste en fonction de la présence d'une liste de réseaux. Elle enregistre les items triés dans deux fichiers différents'''
    # Création des listes
    liste_inclue = []
    liste_exclue = []
    liste_exclue_api = []

    for gare in data:
        # Vérifie la présence d'au moins un network parmis une liste
        if gare["networks"] is None :
            liste_exclue_api.append(gare)
        elif any(reseau_id in [reseau["id"] for reseau in gare["networks"]] for reseau_id in LISTE_RESEAUX_ETUDIES):
            liste_inclue.append(gare)
        else:
            liste_exclue.append(gare)

    #Enregistrement des deux liste après trie
    enregistrer_json(liste_exclue_api,"data/gare_frequentation_exclues_API.json")
    print(f"{len(liste_exclue_api)} gares ont été enregistrées dans data/gare_frequentation_exclues_API.json ")

    enregistrer_json(liste_inclue,"data/gare_frequentation_inclues.json")
    print(f"{len(liste_inclue)} gares ont été enregistrées dans data/gare_frequentation_inclues.json ")

    enregistrer_json(liste_exclue,"data/gare_frequentation_exclues.json")
    print(f"{len(liste_exclue)} gares ont été enregistrées dans data/gare_frequentation_exclues.json ")

In [12]:
def main():
    # Recupération de la clé api depuis env
    load_dotenv()
    api_key = os.getenv("API_KEY")

    #Calcul du nombre de page
    nb_page = calcul_nombre_page()
    #Fais les requetes sur toutes les pages
    data_frequentation_brut = recuperer_toutes_pages(nb_page)
    # Ajout des réseaux à chaque gare
    gares_reseaux = ajout_reseau(data_frequentation_brut,api_key)
    # Trie et enregistrement des gares
    trie_gare(gares_reseaux,LISTE_RESEAUX_ETUDIES)

In [ ]:
main()